In [2]:
# Please install OpenAI SDK first: `pip3 install openai`
from config import DEEPSEEK_API_KEY, OPENROUTER_API_KEY
from openai import OpenAI

client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")#Your model API key here

response = client.chat.completions.create(
    model="deepseek-reasoner",
    timeout= 1200,
    messages=[
        {"role": "system", "content": """You are a helpful assistant. Solve this puzzle for me.
In this puzzle, there are stacks of blocks, and the goal is to rearrange them into a target
configuration using a sequence of moves where:
• Only the topmost block from any stack can be moved.
• A block can be placed either on an empty position or on top of another block.
Example: With initial state [["A", "B"], ["C"], []] and goal state [["A"], ["B"],
["C"]], a solution might be:
moves = [[" C " , 1 , 2] , [" B " , 0 , 1]]
This means: Move block C from stack 1 to stack 2, then move block B from stack 0 to stack 1.
Requirements:
• When exploring potential solutions in your thinking process, always include the corresponding complete list of moves.
• Ensure your final answer also includes the complete list of moves for final solution in the
format: moves = [[block, from stack, to stack], ...]"""},
        {"role": "user", "content":
    """I have a puzzle with 9 blocks.
Initial state:
Stack 0: A, B, C(top)
Stack 1: (empty) (top)
...
Stack 2: (empty) (top)
Goal state:
Stack 0: A (top)
Stack 1: B (top)
...
Stack 2: C (top)
Find the minimum sequence of moves to transform the initial state into the goal state. Remember
that only the topmost block of each stack can be moved."""}
    ],

    stream=False
)

print(response.choices[0].message.content)

The puzzle involves rearranging blocks A, B, and C from the initial state to the goal state using the minimum number of moves. The initial state has Stack 0 containing blocks A, B, and C (with C on top), Stack 1 empty, and Stack 2 empty. The goal state requires Stack 0 to contain only A (with A on top), Stack 1 to contain only B (with B on top), and Stack 2 to contain only C (with C on top).

Only the topmost block of any stack can be moved, and blocks can be placed on an empty stack or on top of another block. Since Stack 1 and Stack 2 are initially empty, they can be used as temporary or final positions.

### Step-by-Step Solution:
1. **Move C from Stack 0 to Stack 2**:  
   - Reason: C is the topmost block on Stack 0 and needs to be on Stack 2 in the goal. Moving it directly to Stack 2 (which is empty) achieves part of the goal efficiently.  
   - State after move:  
     - Stack 0: [A, B] (B is now top)  
     - Stack 1: [] (empty)  
     - Stack 2: [C] (C on top)

2. **Move B from

In [ ]:
from typing import List, Tuple

State = List[List[str]]
Move  = Tuple[str, int, int]      # (block, src_stack, dst_stack)

def validate_moves(moves: List[Move],
                   initial: State,
                   goal: State,
                   verbose: bool = False) -> bool:


    stacks = [col[:] for col in initial]

    def top(idx: int):
        return stacks[idx][-1] if stacks[idx] else None

    for step, (blk, src, dst) in enumerate(moves, start=1):

        if not stacks[src]:
            raise ValueError(f"Step {step}: source stack {src} is empty.")
        if top(src) != blk:
            raise ValueError(
                f"Step {step}: top of stack {src} is {top(src)}, expected {blk}.")


        stacks[src].pop()
        stacks[dst].append(blk)

        if verbose:
            print(f"After step {step}: {stacks}")


    if stacks != goal:
        raise AssertionError(
            f"Final state differs from goal.\nGot : {stacks}\nGoal: {goal}")

    if verbose:
        print("✓ All moves valid and goal reached!")
    return True

if __name__ == "__main__":

    moves = []


    initial_state = []
    goal_state = []


    validate_moves(moves, initial_state, goal_state, verbose=True)